In [13]:
%matplotlib inline
import torch
import numpy as np
import pickle

### Parameters for data generation

In [14]:
N_train_samples = 10000
N_test_samples = 2000

img_size = 2 # 5

#  choose mean
mean_MNpdf = torch.zeros(img_size*img_size)

#choose  covariance, if random the non-diagonal elements are non-zero
random_corr = False

In [15]:
mean_MNpdf.shape

torch.Size([4])

### Prepare for generation

In [16]:
### Create random covariance matrix
if random_corr:
    A =  np.random.rand(img_size*img_size, img_size*img_size)
    B = np.matmul(A, A.transpose())
    eig_vals, eig_vectors = np.linalg.eig(B)
    print(eig_vals[eig_vals <=0].shape)
    
    ## set mean and covariance
    cov_MNpdf = torch.from_numpy(B)
    cov_MNpdf = cov_MNpdf.to(dtype=torch.float32)
else:
    cov_MNpdf =  torch.eye(img_size*img_size)

In [17]:
mean_MNpdf.shape

torch.Size([4])

In [18]:
cov_MNpdf.shape

torch.Size([4, 4])

In [19]:
multiVariateNormalPdf = torch.distributions.MultivariateNormal(loc=mean_MNpdf, covariance_matrix=cov_MNpdf)

In [20]:
multiVariateNormalPdf

MultivariateNormal(loc: torch.Size([4]), covariance_matrix: torch.Size([4, 4]))

In [21]:
multiVariateNormalPdf.sample()

tensor([-0.7372,  0.4012, -0.9561,  0.5465])

### Generate  fake images and save them

In [22]:
#  Sample images from Multivariate Normal  Distribution
labels = {}

partition_list ={'train':[], 'test':[], 'validation':[]}

for i in range(N_train_samples+2*N_test_samples):
    
    name_convention = 'image_'+str(i)
    
    
    X = multiVariateNormalPdf.sample().reshape(img_size, img_size)
    torch.save(X, 'my_data/' + name_convention + '.pt')
    labels[name_convention] = 0
    
    if  i < N_train_samples:
        partition_list['train'].append(name_convention)
    elif (i >= N_train_samples) and (i < N_train_samples + N_test_samples):
        partition_list['validation'].append(name_convention)
    else:
        partition_list['test'].append(name_convention)

In [23]:
pickle.dump([partition_list, labels], open('train_test_info.pkl','wb'))

In [24]:
torch.save([mean_MNpdf, cov_MNpdf], 'MultiVariateNormalParameters.pt')

In [13]:
asfas

NameError: name 'asfas' is not defined

In [ ]:
multiVariateNormalPdf

In [ ]:
from scipy.stats import multivariate_normal

In [ ]:
x = np.linspace(0, 5, 10, endpoint=False)

In [ ]:
x = np.eye(28,28).reshape(1,-1)

In [ ]:
np.zeros((28,1)).shape

In [ ]:
x.shape

In [ ]:
y = multivariate_normal.pdf(x, mean=np.zeros(28*28), cov=np.eye(28*28,28*28));

In [ ]:
y

In [20]:
from scipy.stats import multivariate_normal
dd = 15*15
var = multivariate_normal(mean=np.zeros(dd), cov=np.eye(dd,dd))
np.log(var.pdf(np.zeros(dd)))

-206.76116997105134

In [11]:

torch.transpose(diff, 0, 1)

NameError: name 'diff' is not defined

In [ ]:
img_size

In [42]:
img_size = 28
mean_MNpdf = torch.zeros(img_size*img_size)
cov_MNpdf =  torch.eye(img_size*img_size)

In [46]:
inverse_cov = torch.inverse(cov_MNpdf)
determinant_cov = torch.cholesky(cov_MNpdf).diag().prod()
const_part = -(img_size*img_size)/2 * torch.log(torch.tensor(2*3.141592653)) - 1/2*torch.log(determinant_cov)

def pdf(X):
    X_flattened =  X.flatten()
    diff = (X_flattened - mean_MNpdf)
    pdf_value = torch.matmul( torch.matmul(diff.reshape(1,-1), inverse_cov), diff)[0]
    #print(pdf_value)
    return -1/2*pdf_value + const_part

In [47]:
pdf( torch.ones(img_size*img_size))

tensor(-1112.4478)